# Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import itertools
import os, sys
sys.path.append("..")


from collections import OrderedDict

# Functions from lib

In [2]:
import pandas as pd
import numpy as np

In [3]:
metadata_store = pd.read_csv('/home/jovyan/work/elbaanh/AIO-dev/data/anonimized2/metadata-anon.csv')
# Mixed datatypes in the dimension_name col: floats and OrderedDict as str (need eval)

# Get rid of all irrelevant metadata
metadata_store = metadata_store[ metadata_store.model_type == 'non_seasonal_trend' ]

metadata_store.dimension_name = metadata_store.dimension_name.map(lambda element: eval(element))


<ipython-input-3-46c61e02b491>:1: DtypeWarning: Columns (3,10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_store = pd.read_csv('/home/jovyan/work/elbaanh/AIO-dev/data/anonimized2/metadata-anon.csv')


In [4]:
file = "4weeks-lte_enodeb_id-anon.csv"

In [5]:
dim_name = "lte_enodeb_id" # change based on which file is being read

In [6]:
datas = pd.read_csv( os.path.join("/home/jovyan/work/elbaanh/AIO-dev/data/anonimized2", file) )
datas.drop(labels='Unnamed: 0', axis = 1, inplace = True)
print("-"*30,"DF READ ✔️","-"*30)

------------------------------ DF READ ✔️ ------------------------------


In [7]:
metadata_store = metadata_store[ metadata_store["path"] == file ]

# Converting wide data to long 
Spark needs the data to be in this format for the computations to be parallelized

In [17]:
num = 4000       # Number of timeseries to be in the dataframe for spark processing, change as needed
md_indexes = np.random.choice(metadata_store.index, size = num, replace = False)

datas2_list = []

for _,row in metadata_store.loc[md_indexes].iterrows():
    df = datas[ datas[dim_name] == list(row.dimension_name.values())[0]]
    datas2_list.append( df[[row.kpi_name,"dt","ts",dim_name]] )

In [18]:
datas2 = pd.concat(datas2_list)

In [19]:
# Convert pd data to long format
q = pd.melt(datas2, id_vars = [dim_name, "ts","dt"])
qq = q[[dim_name,"dt","variable","value"]].dropna(subset="value").reset_index(drop=True)
qq.groupby([dim_name, "variable"]).count().index.shape # there is really `num` timeseries in the dataframe

(4000,)

In [195]:
qq.to_parquet(f"/home/jovyan/work/elbaanh/AIO-dev/data/anonimized2/spark/enodeb_datas_long_pa_{num}.parquet")